In [ ]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [ ]:
%%time
!pip uninstall -y torch
!pip install --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl

## Metric

In [ ]:
%%writefile eedi_metrics.py

# Credit: https://www.kaggle.com/code/abdullahmeda/eedi-map-k-metric

import numpy as np
def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## Prepare dataframe

In [ ]:
import os
from transformers import AutoTokenizer
import pandas as pd

IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1).iloc[:500]
df_test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/hugging-quants-meta-llama-3-1-8b-instruct-awq-int4")

PROMPT  = """Question: {Question}
Incorrect Answer: {IncorrectAnswer}
Correct Answer: {CorrectAnswer}
Construct Name: {ConstructName}
Subject Name: {SubjectName}

Your task: Identify the misconception behind Incorrect Answer. Answer concisely and generically inside <response>$$INSERT TEXT HERE$$</response>.
Before answering the question think step by step concisely in 1-2 sentence inside <thinking>$$INSERT TEXT HERE$$</thinking> tag and respond your final misconception inside <response>$$INSERT TEXT HERE$$</response> tag."""

def apply_template(row, tokenizer, targetCol):
    messages = [
        {
            "role": "user", 
            "content": PROMPT.format(
                 ConstructName=row["ConstructName"],
                 SubjectName=row["SubjectName"],
                 Question=row["QuestionText"],
                 IncorrectAnswer=row[f"Answer{targetCol}Text"],
                 CorrectAnswer=row[f"Answer{row.CorrectAnswer}Text"])
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

df = {}
if not IS_SUBMISSION:
    df_label = {}
    for idx, row in df_train.iterrows():
        for option in ["A", "B", "C", "D"]:
            if (row.CorrectAnswer!=option) & (row[f"Misconception{option}Id"]!=-1):
                df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)
                df_label[f"{row.QuestionId}_{option}"] = [row[f"Misconception{option}Id"]]
    df_label = pd.DataFrame([df_label]).T.reset_index()
    df_label.columns = ["QuestionId_Answer", "MisconceptionId"]
    df_label.to_parquet("label.parquet", index=False)
else:
    for idx, row in df_test.iterrows():
        for option in ["A", "B", "C", "D"]:
            if row.CorrectAnswer!=option:
                df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)
df = pd.DataFrame([df]).T.reset_index()
df.columns = ["QuestionId_Answer", "text"]
df.to_parquet("submission.parquet", index=False)

## LLM Reasoning

In [ ]:
%%writefile run_vllm.py

import re
import vllm
import pandas as pd

df = pd.read_parquet("submission.parquet")

llm = vllm.LLM(
    "/kaggle/input/hugging-quants-meta-llama-3-1-8b-instruct-awq-int4",
    quantization="awq",
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=8192,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()


responses = llm.generate(
    df["text"].values,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=2048,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm = True
)

responses = [x.outputs[0].text for x in responses]
df["fullLLMText"] = responses

def extract_response(text):
    return ",".join(re.findall(r"<response>(.*?)</response>", text)).strip()

responses = [extract_response(x) for x in responses]
df["llmMisconception"] = responses
df.to_parquet("submission.parquet", index=False)

In [ ]:
!python run_vllm.py

In [ ]:
llm_output = pd.read_parquet("submission.parquet")

for idx, row in llm_output[-5:].iterrows():
    print(row.fullLLMText)
    print("---"*3)
    print(row.llmMisconception)
    print("==="*6)

In [ ]:
llm_output

## Find similar Misconception

In [ ]:
%%writefile run_similarity_search.py

import pandas as pd
from sentence_transformers import SentenceTransformer, util

df = pd.read_parquet("submission.parquet")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

model = SentenceTransformer('/kaggle/input/bge-large-en-v1-5')
PREFIX = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
input_features = df["text"].str.lstrip(PREFIX).str.split("\n\nYour task:").str[0]

embedding_query = model.encode(input_features+ "\n----\n" + df["fullLLMText"], convert_to_tensor=True)
embedding_Misconception = model.encode(df_misconception_mapping.MisconceptionName.values, convert_to_tensor=True)

top25ids = util.semantic_search(embedding_query, embedding_Misconception, top_k=25)

df["MisconceptionId"] = [" ".join([str(x["corpus_id"]) for x in top25id]) for top25id in top25ids]

df[["QuestionId_Answer", "MisconceptionId"]].to_csv("submission.csv", index=False)

In [ ]:
!python run_similarity_search.py

## Sanity

In [ ]:
import pandas as pd
pd.read_csv("submission.csv")

In [ ]:
if not IS_SUBMISSION:
    import pandas as pd
    from eedi_metrics import mapk
    predicted = pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])
    label = pd.read_parquet("label.parquet")["MisconceptionId"]
    print("Validation: ", mapk(label, predicted))